In [ ]:
# !pip install cdlib
# !pip install infomap
# !pip install wurlitzer
!pip install leidenalg

import networkx as nx
from networkx.algorithms import node_classification
import random
from cdlib import algorithms, evaluation, NodeClustering
from community import community_louvain
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
import math

## 2. Who’s the winner?

### (i)

In [ ]:
def build_synhtetic_graph(mu = 0.5):
    G = nx.Graph(name = "girvan_newman")
    n = 72
    cluster_div = 24
    for i in range(n):
        G.add_node(i, cluster = i // cluster_div + 1)
    for i in range(n):
        for j in range(i + 1, n):
            if G.nodes[i]['cluster'] == G.nodes[j]['cluster']:
                if random.random() < 20 * (1 - mu) / (cluster_div-1):
                    G.add_edge(i, j)
            else:
                if random.random() < 20 * mu / (n-cluster_div):
                    G.add_edge(i, j)
    return G

def compare_on_synthetic(algo_fn, mus, iter_num=10):
    nmis = []
    for mu in mus:
        NMI = 0
        for _ in range(iter_num):
            G = build_synhtetic_graph(mu)
            clustered_G = algo_fn(G)
            partition = get_graph_ideal_partition(G) 
            NMI += clustered_G.normalized_mutual_information(partition).score / iter_num
        nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis
        
def get_graph_ideal_partition(G):
    P = {}
    for node in G.nodes(data = True):
        if node[1]['cluster'] not in P:
            P[node[1]['cluster']] = []
        P[node[1]['cluster']].append(node[0])
    node_clusters = P.values()
    return NodeClustering(list(node_clusters), G, 'Ideal')

def run():
    iter_num = 25
    mus = [0.1 * i for i in range(0, 6)]
    algs = {
        "Louvain": algorithms.louvain,
        "Infomap": algorithms.infomap,
        "Walktrap": algorithms.walktrap
    }
    fig = plt.figure()
    plt.xlabel('µ')
    plt.ylabel('Normalized mutual information')
    plt.xticks(mus)

    for algo_name in algs:
        print('ALGORITHM:', algo_name)
        nmis = compare_on_synthetic(algs[algo_name], mus, iter_num)
        plt.plot(mus, nmis, label=algo_name)
        print('======================'*3)
        
    plt.savefig("images/2_i.jpg")
    plt.legend(loc="best")

run()

### (ii)

In [ ]:
def read_net(folder, graph_name):
    file_name = graph_name + '.net'
    G = nx.MultiGraph(name = file_name)
    with open(os.path.join(folder, file_name), 'r', encoding='utf8') as f:
        f.readline()
        # add nodes
        for line in f:
            if line.startswith("*"):
                break
            else:
                node_info = line.split("\"")
                node = int(node_info[0]) - 1
                label = node_info[1]
                cluster = int(node_info[2]) if len(node_info) > 2 and len(node_info[2].strip()) > 0 else 0
                G.add_node(node, label=label, cluster=cluster)

        # add edges
        for line in f:
            node1_str, node2_str = line.split()[:2]
            G.add_edge(int(node1_str)-1, int(node2_str)-1)
    return G

def compare_on_lrf(algo_fn, mus, iter_num=10):
    nmis = []
    for mu in mus:
        NMI = 0
        for i in range(iter_num):
            G = read_net("data/LFR/", "LFR_0{}_{}".format(int(mu*10), i))
            clustered_G = algo_fn(G)
            partition = get_graph_ideal_partition(G) 
            NMI += clustered_G.normalized_mutual_information(partition).score / iter_num
        nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis
        
def get_graph_ideal_partition(G):
    P = {}
    for node in G.nodes(data = True):
        if node[1]['cluster'] not in P:
            P[node[1]['cluster']] = []
        P[node[1]['cluster']].append(node[0])
    node_clusters = P.values()
    return NodeClustering(list(node_clusters), G, 'Ideal')

def run():
    iter_num = 25
    mus = [0.2 * i for i in range(0, 5)]
    algs = {
        "Louvain": algorithms.louvain,
        "Infomap": algorithms.infomap,
        "Walktrap": algorithms.walktrap
    }
    fig = plt.figure()
    plt.xlabel('µ')
    plt.ylabel('Normalized mutual information')
    plt.xticks(mus)

    for algo_name in algs:
        print('ALGORITHM:', algo_name)
        nmis = compare_on_lrf(algs[algo_name], mus, iter_num)
        plt.plot(mus, nmis, label=algo_name)
        print('======================'*3)
        
    plt.legend(loc="best")
    plt.savefig("images/2_ii.jpg")

run()

### (iii)

In [ ]:
def compare_on_random(algo_fn, avg_degrees, iter_num=10):
    nvis = []
    n = 1000
    for avg_degree in avg_degrees:
        NVI = 0
        for i in range(iter_num):
            G = nx.gnm_random_graph(n, avg_degree*n/2)
            clustered_G = algo_fn(G)
            partition = NodeClustering([[r] for r in range(n)], graph=G, method_name="rand_graph")
            NVI += clustered_G.variation_of_information(partition).score / math.log(n)
        NVI /= iter_num
        nvis.append(NVI)
        print("avg degree={:.2f}, NVI: {:5.3f}".format(avg_degree, NVI))
    return nvis

def run():
    iter_num = 25
    avg_degrees = [8 * i for i in range(1, 6)]
    algs = {
        "Louvain": algorithms.louvain,
        "Infomap": algorithms.infomap,
        "Walktrap": algorithms.walktrap
    }
    fig = plt.figure()
    plt.xlabel('µ')
    plt.ylabel('Normalized variation of information')
    plt.xticks(avg_degrees)

    for algo_name in algs:
        print('ALGORITHM:', algo_name)
        nvis = compare_on_random(algs[algo_name], avg_degrees, iter_num)
        plt.plot(avg_degrees, nvis, label=algo_name)
        print('======================'*3)
        
    plt.legend(loc="best")
    plt.savefig("images/2_iii.jpg")

run()

### (iv)

In [ ]:
def compare_on_dolphins(G, algo_fn, iter_num=25):
    N = G.number_of_nodes()
    communities = []
    for i in range(iter_num):
        clustered_G = algo_fn(G)
        communities.append(clustered_G)
    NVI = 0
    for i in range(iter_num):
        for j in range(i+1, iter_num):
            NVI += communities[i].variation_of_information(communities[j]).score / math.log(N)
    NVI /= iter_num*(iter_num-1)/2
    print("NVI: {:5.3f}".format(NVI))
    return NVI

def run():
    iter_num = 25
    G = read_net("data", "dolphins")
    algs = {
        "Louvain": algorithms.louvain,
        "Infomap": algorithms.infomap,
        "Walktrap": algorithms.walktrap
    }
    for algo_name in algs:
        print('ALGORITHM:', algo_name)
        nmi = compare_on_dolphins(G, algs[algo_name], iter_num)
        print('======================'*3)

run()

## 3. Peers, ties and the Internet

In [ ]:
def leiden(G, node_pairs):
    coms = algorithms.leiden(G)
    C = dict()
    for i in range(len(coms.communities)):
        for node in coms.communities[i]:
            C[node] = i
    part_G = community_louvain.induced_graph(C, G)

    scores = []
    for n1, n2 in node_pairs:
        score = 0
        if C[n1] == C[n2]:
            try: 
                nc = len(coms.communities[C[n1]])
                mc = part_G[C[n1]][C[n2]]['weight']
                score = mc / (nc * (nc-1) / 2)
            except KeyError:
                score = 0
        scores.append([n1, n2, score])
    return scores

def sample(G, p):
    nodes = list(G.nodes())
    N = int(G.number_of_edges()*p)
    
    Ln = []
    while len(Ln) != N:
        node1, node2 = random.choice(nodes), random.choice(nodes)
        if not G.has_edge(node1, node2) and (node1, node2) not in Ln:
            Ln.append((node1, node2))
    
    Lp = random.sample(list(G.edges()), N)
    G.remove_edges_from(Lp)  
    
    return G, [*Ln, *Lp], len(Ln), N

def AUC(G, algs):
    G, nodes, l, N = sample(G.copy(), 0.1)

    scores = []
    for algo_name in algs:
        predicted = list(algs[algo_name](G, nodes))
        m1, m2 = 0, 0
        for n in range(N):
            s1 = random.sample(predicted[:l], 1)[0]
            s2 = random.sample(predicted[l:], 1)[0]
            if s2[2] > s1[2]:
                m1 += 1
            elif s1[2] == s2[2]:
                m2 +=1
        scores.append((m1 + m2/2) / N)
    return np.array(scores)

def AUC_runs(G, name, algs, n=10):
    auc_l, auc_p, auc_a = 0, 0, 0
    for i in range(n):
        scores = AUC(G, algs)
        auc_l += scores[0]
        auc_p += scores[1]
        auc_a += scores[2]
    print(name + ':')
    print("  Leiden:", auc_l/n)
    print("  Preferential attachment:", auc_p/n)
    print("  Adamic adar index:", auc_a/n)

In [ ]:
algs = {
    "Leiden": leiden,
    "Pref attachment": nx.preferential_attachment,
    "Adamic adar index": nx.adamic_adar_index
}
G = nx.Graph(nx.read_pajek("data/gnutella.net")).to_undirected()
AUC_runs(G, "Gnutella", algs, 10)

In [ ]:
G = nx.Graph(nx.read_pajek("data/circles.net")).to_undirected()
AUC_runs(G, "Facebook", algs, 10)

In [ ]:
G = nx.Graph(nx.read_pajek("data/nec.net")).to_undirected()
AUC_runs(G, "Internet", algs, 10)

In [ ]:
G = nx.gnm_random_graph(25000, 125000)
AUC_runs(G, "Random", algs, 10)

## 4. Get at least 70% right!

In [ ]:
def read_and_split(filepath):
    G = nx.Graph() 

    with open(filepath, 'r') as f:
    f.readline()
    test = []
    for line in f:
        if line.startswith("*"):
            continue

        l = line.split()
        if len(l) == 3:
        if l[1][-3:-1] == '13':
            G.add_node(int(l[0]))
            test.append([int(l[0]), l[2]])
        else:
            G.add_node(int(l[0]), label=l[2])
        else:
        G.add_edge(int(l[0]), int(l[1]))

    return G, test

In [ ]:
G, test = read_and_split("data/aps_2008_2013.net")
iter_num = 10
acc = 0
for _ in range(iter_num):
    #predicted = node_classification.harmonic_function(G)
    predicted = node_classification.local_and_global_consistency(G)
    count = 0
    for (node, label) in test:
        if predicted[node-1] == label:
        count += 1

    acc += count
acc /= len(test)*iter_num
print("Accuracy:", acc)